In [1]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    out_Seq = ''
    
    
    #Genome folder
    entries = os.listdir("/mnt/f/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = open(("/mnt/f/Genomes_2023-12-26/"+Species+"/"+Genome_name),'r')
    
    #Extract the sequence 
    for record in SeqIO.parse(fasta_file,"fasta"):
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = out_Seq.reverse_complement()
                
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    fasta_file.close()
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)
#

In [2]:
import os
from Bio import SeqIO

In [4]:
#%reset -f
blast_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/6.Period Exon Analysis/7.Nymphalinae"
query_location = f"{blast_location}/1.Query"
query_species = "20.Vanessa_tameamea"
query_transcript_list = os.listdir(f"{query_location}/{query_species}")
if "desktop.ini" in query_transcript_list:
    query_transcript_list.remove("desktop.ini")

query_transcript = query_transcript_list[0]
#Get Species list, the list may be a list of files or a single species
# query_species = "22.Pieris_brassicae"
# phylo_species = open("C:/Users/sauba/Desktop/Work_Stuff/MRJP/0.for_automation_temp_files/phylo_species.txt",'r')
# species_list = phylo_species.readlines()
# species_list = ["10.Plutella_xylostella","11.Bombyx_mori","12.Manduca_sexta","13.Danaus_plexippus","14.Zerene_cesonia","15.Hyposmocoma_kahamanoa","16.Papilio_xuthus","17.Papilio_polytes","18.Bombyx_mandarina","19.Trichoplusia_ni","2.Galleria_mellonella","20.Vanessa_tameamea","21.Amyelois_transitella","22.Pieris_brassicae","23.Vanessa_atalanta","24.Spodoptera_frugiperda","25.Pieris_napi","26.Vanessa_cardui","27.Maniola_hyperantus","28.Maniola_jurtina","29.Nymphalis_io","3.Spodoptera_litura","30.Leptidea_sinapis","31.Leguminivora_glycinivorella","32.Colias_croceus","33.Pararge_aegeria","36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
species_list = os.listdir(f"{blast_location}/1.Blast_result")
species_list.remove('desktop.ini')
# species_list = ["5.Bicyclus_anynana"]
# species_list = ["29.Nymphalis_io","3.Spodoptera_litura","30.Leptidea_sinapis","31.Leguminivora_glycinivorella","32.Colias_croceus","33.Pararge_aegeria","36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
# species_list = ["36.Melitaea_cinxia","37.Aricia_agestis","38.Papilio_machaon","39.Leptidea_sinapis","4.Helicoverpa_armigera","40.Colias_croceus","41.Manduca_sexta","42.Spodoptera_frugiperda","43.Ostrinia_furnacalis","5.Bicyclus_anynana","6.Pectinophora_gossypiella","7.Helicoverpa_zea","8.Plodia_interpunctella","9.Pieris_rapae"]
#print(species_list)
# species_list = ["3.Spodoptera_litura"]
#get list of query names

query_file = SeqIO.parse(f"{blast_location}/1.Query/{query_species}/{query_transcript}/query_protein.fa", 'fasta')
query_name_list = []

for records in query_file:
    query_name_list.append(records.id)



for species in species_list:
    # print(species_name)
    # species = species_name.split(".")[1]
    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    
#Run for each query in the query list
    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
#         print(f"Query name = {query_name}")
        Length_switch = "0"
        with open(f"{blast_location}/1.Blast_result/{species}/{species}_blast_out.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()
        #print(lines_in_file)

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0

        for lines in lines_in_file:

#             print(lines)
            if query_name in lines:
#                 print(lines)
#                 assert False
            #Initialize that results can now be checked
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
                query_species = str(query_species_split[0].split(".")[1]+"_"+query_species_split[1].rstrip())
                print(query_species)
            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            length = 0
            start = 000
            stop = 000
            query_start_coor = 000
            query_stop_coor = 000
            error = "Y"
            complement  = 0
        
        
        
        

        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''
#             print(start, stop)


#Check for Met or STOP at the start and end
        while True and (start != 0 or stop != 0):
            if (query_name == query_name_list[0] or query_name == query_name_list[-1]):
                frame = 1
                translated_sequence = (sequence_extractor(species, scaff, int(complement), start, stop,frame ))

                if translated_sequence == old_trans:
                    break
                    
#Check for Met at the beginning of first exon

                if query_name == query_name_list[0] :
                    if translated_sequence[0] != "M":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            start = int(start) - 3
                        if complement == "1":
                            stop = int(stop) + 3
                        old_trans = translated_sequence
                    if "*" in translated_sequence:
                        error = "Y"
                        break
                    if translated_sequence[0] == "M":
                        break

#Check for stop at the end of last exon

                if query_name == query_name_list[-1]:
                    if translated_sequence[-1] != "*":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            stop = int(stop) + 3
                        if complement == "1":
                            start = int(start) - 3
                        old_trans = translated_sequence
                    if translated_sequence[-1] == "*":

                        break
            else:
                break
        
        
#Add offset at the beginning and end of each exon
        if (start != 0 or stop != 0):
            
            start_modifier = int(query_name.split("Frame")[1][1])
            stop_modifier = int(query_name.split("rightoh")[1][1])
        else:
            start_modifier = 0
            stop_modifier = 0            

        seq_length = query_length
        
        
#Get exon start and end by adding or removing codons
        
    #if not the beginning
        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"
            
           
        output_format = str(species.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        print(output_format)
#         assert False
        Output_Sequence = Output_Sequence + output_format

    print(Output_Sequence)
   
    output_file = open(f"{blast_location}/1.Blast_result/{species}/{species}_coordinates_old.csv",'w')
    output_file.write(Output_Sequence)
    output_file.close()


Vanessa_tameamea
Euphydryas_editha,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_2_query_Exon_1,0,0,31

Vanessa_tameamea
Euphydryas_editha,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_2,0,0,27

Vanessa_tameamea
Euphydryas_editha,CAKOGL010000026.1,15663855,15663962,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_3,6,35,35

Vanessa_tameamea
Euphydryas_editha,CAKOGL010000026.1,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_4,0,0,27

Vanessa_tameamea
Euphydryas_editha,CAKOGL010000026.1,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_1_query_Exon_5,0,0,54

Vanessa_tameamea
Euphydryas_editha,CAKOGL010000012.1,13991542,13991639,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_0_query_Exon_6,3,23,32

Vanessa_tameamea
Euphydryas_editha,CAKOGL010000012.1,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_7,0,0,25

Vanessa_tameamea
Euphydryas_editha,

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,279953,280084,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_8,4,44,44

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,280306,280366,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_9,1,20,20

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,283625,283828,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_10,1,67,67

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,285111,285339,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_2_query_Exon_11,2,75,75

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,285853,285964,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_12,1,37,37

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,288183,288302,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_13,1,40,40

Vanessa_tameamea
Hypolimnas_misippus,SGBE01000429.1,288773,288922,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_que

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9958845,9958976,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_8,1,44,44

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9958238,9958298,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_9,1,20,20

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9957566,9957769,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_10,7,67,67

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9956877,9957105,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_2_query_Exon_11,2,75,75

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9956687,9956798,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_12,1,37,37

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9956068,9956190,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_13,1,40,40

Vanessa_tameamea
Melitaea_cinxia,NC_059424.1,9955376,9955525,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_14,1,52,52

Vanessa_tameame

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10946815,10946964,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_14,1,52,52

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10946273,10946391,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_2_query_Exon_15,2,39,39

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10945588,10945665,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_16,1,23,25

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10945303,10945449,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_17,1,49,49

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10944546,10944717,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_18,1,57,57

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10944100,10944238,1,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_19,2,34,46

Vanessa_tameamea
Mellicta_athalia,HG992176.1,10943563,10943664,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_20,1,32,34


Vanessa_tameamea
Nymphalis_c-album,OY720586.1,12329611,12329712,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_26,1,34,34

Vanessa_tameamea
Nymphalis_c-album,OY720586.1,12329128,12329268,1,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_27,1,26,47

Vanessa_tameamea
IRRRKREYSKLVMIYEEDAPLPSADEEEA
IRRRKREYSKLVMIYEEDAPLPSADEEEAA
Nymphalis_c-album,OY720586.1,12328439,12328531,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_28,1,29,34

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Nymphalis_c-album,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_2_query_Exon_1,0,0,31
Nymphalis_c-album,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_2,0,0,27
Nymphalis_c-album,OY720586.1,12349686,12349793,1,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_3,6,30,35
Nymphalis_c-album,OY720586.1,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5797949,5798098,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_17,10,49,49

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5798808,5798979,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_18,1,57,57

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5799868,5800006,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_19,2,43,46

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5801583,5801687,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_20,1,33,34

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5803052,5803165,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_21,1,36,38

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5806724,5806878,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_0_query_Exon_22,1,50,50

Vanessa_tameamea
Nymphalis_io,NC_065918.1,5809322,5809456,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_23,1,31,45

Vanessa_tameamea
Nymphalis_io,NC_

MRRRKREYSKLVMIYEEDAPLPSPDD
MRRRKREYSKLVMIYEEDAPLPSPDDG
MRRRKREYSKLVMIYEEDAPLPSPDDGQ
MRRRKREYSKLVMIYEEDAPLPSPDDGQS
MRRRKREYSKLVMIYEEDAPLPSPDDGQSF
MRRRKREYSKLVMIYEEDAPLPSPDDGQSFI
MRRRKREYSKLVMIYEEDAPLPSPDDGQSFIV
Nymphalis_polychloros,HG992241.1,12164474,12164572,1,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_28,1,26,34

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Nymphalis_polychloros,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_2_query_Exon_1,0,0,31
Nymphalis_polychloros,Intial_value,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_2,0,0,27
Nymphalis_polychloros,HG992241.1,12188873,12188980,1,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_3,6,29,35
Nymphalis_polychloros,HG992241.1,0,0,0,Y,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_4,0,0,27
Nymphalis_polychloros,HG992241.1,12188133,12188296,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_

Vanessa_atalanta,NC_061902.1,10979923,10980017,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_2_query_Exon_1,1,31,31

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10977557,10977640,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_2,1,27,27

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10975630,10975737,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_3,1,35,35

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10975252,10975335,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_2_query_Exon_4,1,27,27

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10974627,10974790,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_1_query_Exon_5,1,54,54

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10973133,10973230,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_0_query_Exon_6,1,32,32

Vanessa_tameamea
Vanessa_atalanta,NC_061902.1,10972737,10972811,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_7,1,25,25

Vanessa_tameamea

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11466523,11466597,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_7,1,25,25

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11465649,11465780,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_8,1,44,44

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11464727,11464787,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_9,1,20,20

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11462475,11462678,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_10,1,67,67

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11461557,11461785,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_2_query_Exon_11,1,75,75

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11460923,11461034,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_12,1,37,37

Vanessa_tameamea
Vanessa_cardui,NC_061154.1,11460673,11460792,1,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_13,1,40,40

Vanessa_t

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,26558,26729,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_1_rightoh_0_query_Exon_18,1,57,57

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,27371,27509,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_1_query_Exon_19,1,46,46

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,27800,27904,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_20,1,34,34

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,28440,28556,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_1_query_Exon_21,1,38,38

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,29248,29399,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_2_rightoh_0_query_Exon_22,1,50,50

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,31995,32129,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_23,1,45,45

Vanessa_tameamea
Vanessa_tameamea,NW_020662887.1,33545,33727,0,N,20.Vanessa_tameamea_XM_026643498.1_Frame_0_rightoh_0_query_Exon_24,1,61,61

Vanessa_tamea